For joining a large DataFrame with a small one, broadcast the smaller one to avoid shuffles.

In [0]:
from pyspark.sql.functions import expr, broadcast

# Simulate large and small tables
df_large = spark.range(1_000_000).withColumn("key", expr("id % 100"))
df_small = spark.range(100).withColumnRenamed("id", "key").withColumn("value", expr("key * 10"))

# Broadcast the small DataFrame
from pyspark.sql.functions import broadcast

df_joined = df_large.join(broadcast(df_small), on="key", how="inner")
df_joined.show()


Without Broadcast

In [0]:
from pyspark.sql.functions import expr, broadcast

# Simulate large and small tables
df_large = spark.range(1_000_000).withColumn("key", expr("id % 100"))
df_small = spark.range(100).withColumnRenamed("id", "key").withColumn("value", expr("key * 10"))

# Broadcast the small DataFrame
from pyspark.sql.functions import broadcast

df_joined = df_large.join(df_small, on="key", how="inner")
df_joined.show()

Validating Query Plan

In [0]:
from pyspark.sql.functions import broadcast

df_large = spark.range(1_000_000).withColumn("key", (expr("id % 100")))
df_small = spark.range(100).withColumnRenamed("id", "key")

# Without broadcast
df_large.join(df_small, "key").explain(extended=True)

# With broadcast
df_large.join(broadcast(df_small), "key").explain(extended=True)


Query Type	Description
Query A (Implicit)	Regular join on key, no broadcast hint.
Query B (Explicit)	Same join with BROADCAST() hint on right side.

What We Observe from the Physical Plan
Both queries used:
PhotonBroadcastHashJoin → Broadcast join was applied successfully.

BuildRight → The right table (small one) is broadcasted.

PhotonRange and PhotonProject → Photon engine is vectorizing every stage.

PhotonShuffleExchangeSink → Small dataset is still shuffled, which seems counterintuitive.






Why Is There Still a Shuffle?
Even though Spark broadcasts the small table:

It may use PhotonShuffleExchangeSink to distribute the broadcasted data across nodes during physical planning.

This happens especially when the small dataset is partitioned across multiple executors, and Photon parallelizes the broadcast phase.

The presence of PhotonShuffleExchangeSink in a broadcast join doesn't mean the large table is shuffled — it's just how Photon handles partitioned inputs efficiently.